In [4]:
from bs4 import BeautifulSoup
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from time import sleep
from tqdm import tqdm
import pandas as pd

s = Service("./chromedriver")
browser = Chrome(service = s)
url = "https://www.kinopoisk.ru/lists/top250/"
browser.get(url)

sleep(15) # time for captch passing
 
data = []

for i in tqdm(range(5)):
    sleep(7)
    soup = BeautifulSoup(browser.page_source, 'lxml')
    films = soup.findAll("div", class_ = "desktop-rating-selection-film-item")
    
    for film in films:
        try:
            link = "kinopoisk.ru" + film.find("a", class_="selection-film-item-meta__link").get("href")
        except:
            link = ""
        try:
            russian_name = film.find("a", class_="selection-film-item-meta__link").find("p", class_="selection-film-item-meta__name").text
        except:
            russian_name = ""
        try:
            original_name = film.find("a", class_="selection-film-item-meta__link").find("p", class_="selection-film-item-meta__original-name").text
        except:
            original_name = ""
        try:
            country = film.find("a", class_="selection-film-item-meta__link").find("span", class_="selection-film-item-meta__meta-additional-item").text
        except:
            country = ""
        try:
            film_type = film.find("a", class_="selection-film-item-meta__link").findAll("span", class_="selection-film-item-meta__meta-additional-item")[1].text
        except:
            film_type = ""
        try:
            rate = film.find("span", class_ = "rating__value rating__value_positive").text
        except:
            rate = ""
        data.append([link, russian_name, original_name, country, film_type, rate])
    try:
        browser.find_elements(By.CLASS_NAME, "paginator__page-relative")[-1].click()
        # On all pages, except the first one, there are two buttons with such class: "Previous" and "Next" page,
        # - so we are going to find the "Next" button in all cases, which is always the last one
        # - 1 of 1 on the first page and 2 of 2 on others
    except:
        break
        
header = ["link", "russian_name", "original_name", "country", "film_type", "rate"]
df = pd.DataFrame(data, columns = header)
df.to_csv("./kinopoisk_data.csv", sep = ";", encoding = "utf8")

100%|██████████| 5/5 [00:36<00:00,  7.25s/it]
